In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
#import 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd 
import bs4 as bs
import nltk
from nltk.tokenize import sent_tokenize # tokenizes sentences
import re
from nltk.stem import PorterStemmer
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

#Machine Learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics # for confusion matrix, accuracy score etc
from sklearn.model_selection import train_test_split

In [2]:
from data_prep import DataPreprocessing as DP

In [3]:
df = DP.data_preprocessing()

In [4]:
df

,_unit_state,choose_one_category,tweet_text,cat
0,finalized,other_useful_information,Cyclone Pam hits New Zealand - dominion-post |...,storm
1,finalized,infrastructure_and_utilities_damage,#CylonePam devastates #Vanuatu: Regional disas...,storm
2,finalized,injured_or_dead_people,RT @9NewsAUS: Emergency response teams from Au...,storm
3,finalized,other_useful_information,Vanuatu â°Â_ #RT Cyclone Pam Lashes Vanuatu ...,storm
4,finalized,other_useful_information,Aid agencies: Vanuatu conditions more challeng...,storm
...,...,...,...,...
2008,golden,other_useful_information,An earthquake creates new island in #Pakistan!...,eq
2009,golden,not_related_or_irrelevant,RT @sairabaig: Rt@oOol_JaLoOl: Tabaaahi dar Ta...,eq
2010,golden,other_useful_information,RT @FoxNews: New massive earthquake rocks sout...,eq
2011,golden,injured_or_dead_people,20 more bodies recovered from a seminary in Ba...,eq


In [5]:
#from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, StandardScaler
#from sklearn.impute import SimpleImputer
from sklearn.pipeline import FeatureUnion, Pipeline 

In [15]:
X = df['tweet_text']
y = df['choose_one_category'].values

In [16]:
X_train, X_test, y_train, y_test = train_test_split( X, y , test_size = 0.2 , random_state = 42 )

In [17]:
X_train

764     Dude. Disease happens. You have #ObamaDerangem...
979     10 Questions on the Deadly Middle Eastern Viru...
286     RT @globalnews: Government says at least 29 Ca...
342     Chile Earthquake Offers Reminder That U.S. Fac...
1624    Army evacuates 20,000 people affected by flood...
                              ...                        
891     Nepal and India begin relief efforts as monsoo...
443     Ã£â¬Â#USGS #alertÃ£â¬â M 2.0, Central Ala...
359     Philippines residents flee as Typhoon Hagupit ...
860     Lola is on cyclone watch! #bringthatrain! http...
706     FOLLOW @iat_music EBOLA VIRUS FACTS AND FALLAC...
Name: tweet_text, Length: 18516, dtype: object

In [8]:
from CT import OwnTransformer as OT

In [20]:
ot = OT()

In [21]:
test_pipeline = Pipeline(steps=[('text_cleaner', ot)])

In [ ]:
df

In [22]:
X_clean = test_pipeline.fit(X_train, y_train)

In [23]:
X_train

764     Dude. Disease happens. You have #ObamaDerangem...
979     10 Questions on the Deadly Middle Eastern Viru...
286     RT @globalnews: Government says at least 29 Ca...
342     Chile Earthquake Offers Reminder That U.S. Fac...
1624    Army evacuates 20,000 people affected by flood...
                              ...                        
891     Nepal and India begin relief efforts as monsoo...
443     Ã£â¬Â#USGS #alertÃ£â¬â M 2.0, Central Ala...
359     Philippines residents flee as Typhoon Hagupit ...
860     Lola is on cyclone watch! #bringthatrain! http...
706     FOLLOW @iat_music EBOLA VIRUS FACTS AND FALLAC...
Name: tweet_text, Length: 18516, dtype: object

In [ ]:
# df_clean method default settings hashtag_just_sign=True, remove_stopwords=True, test_position=4
df_clean = tweet_cleaner(df, remove_stopwords=True, hashtag_just_sign = False, tp_start=100, tp_end=150)

train on content: 'other_useful_information', 'infrastructure_and_utilities_damage',
       'injured_or_dead_people', 'not_related_or_irrelevant',
       'donation_needs_or_offers_or_volunteering_services',
       'caution_and_advice', 'sympathy_and_emotional_support',
       'missing_trapped_or_found_people',
       'displaced_people_and_evacuations', 'affected_people',
       'disease_signs_or_symptoms', 'prevention', 'disease_transmission',
       'treatment', 'deaths_reports'

In [ ]:
output_content, output_content_importances, output_content_feature_names = training_on_content(df_clean)

train on cat: 'storm', 'eq', 'flood', 'virus'

In [ ]:
output_category, output_cat_importances, output_cat_feature_names = training_on_cat(df_clean)

## Output the lists

In [ ]:
print(output_content)
print(output_cat_importances)
#control output that we put out the identical list in both cases
print(output_cat_feature_names[1][100:110])

In [ ]:
print(output)
print(output_content_importances)
#control output that we put out the identical list in both cases
print(output_content_feature_names[1][100:110])

# Import Twitter datasets

source data from https://crisisnlp.qcri.org/lrec2016/lrec2016.html

data presented by:
- Muhammad Imran, Prasenjit Mitra, Carlos Castillo: Twitter as a Lifeline: Human-annotated Twitter Corpora for NLP of Crisis-related Messages. In Proceedings of the 10th Language Resources and Evaluation Conference (LREC), pp. 1638-1643. May 2016, Portorož, Slovenia.

### cleaning steps

In [ ]:
def tweet_cleaner (dfc, hashtag_just_sign=True, remove_stopwords=True, tp_start=4, tp_end=10, cleaner='self_made'):  
    list_before = list()
    list_after = list()
    
    #check one example before cleaning
    print('example before cleaning at positions ', tp_start, ' to ', tp_end)
    #for row in (dfc.iloc[tp_start:tp_end]).rows:
    #   print(row['tweet_text'], '  ---  ',row['choose_one_category'])

    for index, row in dfc.iloc[tp_start:tp_end].iterrows():
        list_before.append([row['tweet_text'], row['choose_one_category']])
        
    if(cleaner=='off_the_shelf'):
        print('Not defined pre-processor, can\'t be used')
    
    if(cleaner=='self_made'):
        #IN ANY CASE 
        #non alpabetical/ numerical
        dfc = dfc.replace(to_replace =r'&amp;', value = '', regex = True)
        dfc = dfc.replace(to_replace =r'&gt;', value = '', regex = True)
        #hyperlinks
        dfc = dfc.replace(to_replace =r'http\S+', value = '', regex = True)
        #usernames
        dfc = dfc.replace(to_replace =r'@\S+', value = '', regex = True) 
        #remove retweet
        dfc = dfc.replace(to_replace ='RT :', value = '', regex = True) 
        dfc = dfc.replace(to_replace ='RT ', value = '', regex = True)


        #HASHTAG OPTIONAL REMOVE
        if(hashtag_just_sign == True):
            dfc = dfc.replace(to_replace ='#', value = '', regex = True) 
        else:
            dfc = dfc.replace(to_replace =r'#\S+', value = '', regex = True)

        #IN ANY CASE
        #remove punctation
        dfc = dfc.replace(to_replace ='[",:!?\\-]', value = ' ', regex = True)
        #4. Tokenize into words (all lower case)
        dfc.tweet_text = dfc.tweet_text.str.lower()
        #make sure no weird letters left

        #u = df.select_dtypes(object)
        dfc['tweet_text'] = dfc['tweet_text'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))

        if(remove_stopwords == True):
            dfc.tweet_text = dfc.tweet_text.str.split() 
            eng_stopwords = set(stopwords.words("english"))
            dfc['tweet_text'] = dfc['tweet_text'].apply(lambda x: [item for item in x if item not in eng_stopwords])
            #join the list items back to one string
            dfc['tweet_text'] = dfc['tweet_text'].apply(lambda x: ' '.join(x))

    
    
    #print('example after cleaning at positions ', tp_start, ' to ', tp_end)
    #print(dfc.iloc[tp_start:tp_end].tweet_text)
    
    for index, row in dfc.iloc[tp_start:tp_end].iterrows():
        list_after.append([row['tweet_text'], row['choose_one_category']])
    
    
    for num in range(0, len(list_before)):
        print(list_before[num])
        print(list_after[num])
        print('--------------------------------------------------------------')
    
    return dfc

## Classify dataset

In [ ]:
#file=df_clean.copy(deep=True)

In [ ]:
def training_on_content(file):
    content = list(file.choose_one_category.unique())
    content_importances = list()
    content_feature_names = list()

    for con in content:
        #build the needed dataset
        temp_file = file.copy(deep=True)
        replacing_list = list()
        replacing_list = content.copy()
        replacing_list.remove(con)
        #print(replacing_list)
        string_to_be_added = str('no_'+ con)
        #print(string_to_be_added)
        print('testing ...')
        #print(replacing_list)
        temp_file.choose_one_category = temp_file.choose_one_category.replace(to_replace=replacing_list, value=string_to_be_added) 
        print('                         ')
        print('Train forest for ', con)
        print('---------------------------')
        forest_out, vectorizer_out, feature_names_out, importances_out = train_forest(temp_file['tweet_text'], temp_file['choose_one_category'])
        content_feature_names.append(feature_names_out)
        content_importances.append(importances_out)

    print('DONE with Training on Content!!!')
    
    return conent, content_importances, content_feature_names

In [ ]:
def training_on_cat(file):    
    category = list(file.cat.unique())
    cat_importances = list()
    cat_feature_names = list()

    for cat in category:
        #build the needed dataset
        temp_file = file.copy(deep=True)
        replacing_list = list()
        replacing_list = category.copy()
        replacing_list.remove(cat)
        #print(replacing_list)
        string_to_be_added = str('no_'+ cat)
        #print(string_to_be_added)
        print('testing ...')
        #print(replacing_list)
        temp_file.cat = temp_file.cat.replace(to_replace=replacing_list, value=string_to_be_added) 
        print('                         ')
        print('Train forest for ', cat)
        print('---------------------------')
        forest_out, vectorizer_out, feature_names_out, importances_out = train_forest(temp_file['tweet_text'], temp_file['cat'])
        cat_feature_names.append(feature_names_out)
        cat_importances.append(importances_out)

    print('DONE with Training on Cat!!!')
    
    return content, cat_importances, cat_feature_names

# Random Forest Training Class

In [ ]:
def train_forest(data, y):

    print("Creating the bag of words model!\n")
    # CountVectorizer" is scikit-learn's bag of words tool, here we show more keywords 
    vectorizer = CountVectorizer(analyzer = "word",   \
                                 tokenizer = None,    \
                                 preprocessor = None, \
                                 stop_words = None,   \
                                 max_features = 2000) 
    
    X_train, X_test, y_train, y_test = train_test_split(\
    data, y, random_state=0, test_size=.2)

    # Then we use fit_transform() to fit the model / learn the vocabulary,
    # then transform the data into feature vectors.
    # The input should be a list of strings. .toarraty() converts to a numpy array
    
    train_bag = vectorizer.fit_transform(X_train).toarray()
    test_bag = vectorizer.transform(X_test).toarray()

    # You can extract the vocabulary created by CountVectorizer
    # by running print(vectorizer.get_feature_names())


    print("Training the random forest classifier!\n")
    # Initialize a Random Forest classifier with 75 trees
    forest = RandomForestClassifier(n_estimators = 50) 

    # Fit the forest to the training set, using the bag of words as 
    # features and the sentiment labels as the target variable
    forest = forest.fit(train_bag, y_train)

    train_predictions = forest.predict(train_bag)
    test_predictions = forest.predict(test_bag)
    
    train_acc = metrics.accuracy_score(y_train, train_predictions)
    valid_acc = metrics.accuracy_score(y_test, test_predictions)
    print("The training accuracy is: ", train_acc, "\n", "The validation accuracy is: ", valid_acc)
    
    print('confusion matrix')
    print(metrics.confusion_matrix(y_test,test_predictions))
    
    print('check classes on which we trained')
    print(forest.classes_)
    
    importances = forest.feature_importances_
    # returns relative importance of all features.
    # they are in the order of the columns
    print(importances)
    len(importances)

    feature_names = vectorizer.get_feature_names()
    # sort importance scores
    
    indices = np.argsort(importances)[::-1]
    # Print the feature ranking
    print("Feature ranking:")
    top_50 = indices[:50]
    top50_features = [vectorizer.get_feature_names()[ind] for ind in top_50]
    print(top50_features)
    
    return(forest, vectorizer, feature_names, importances)